In [3]:
# https://zhuanlan.zhihu.com/p/419092667
import torch
import numpy

class SVM(torch.nn.Module):
    def __init__(self, C = 1):
        super(SVM,self).__init__()
        self.w = torch.nn.Parameter(torch.ones(4096))
        self.b = torch.nn.Parameter(torch.zeros(1))
        # C正相关越界点的个数
        self.C = C
        self.relu = torch.nn.ReLU()

    def forward(self, positive, negative, optimizer):
        optimizer.zero_grad()
        w2 = torch.pow(self.w, 2).sum()
        positive = 1 - (torch.matmul(positive, self.w) + self.b)    # 正类
        negative = 1 + (torch.matmul(negative, self.w) + self.b)    # 负类

        loss = (self.relu(positive).sum() + self.relu(negative).sum()) / self.C + w2

        loss.backward()
        optimizer.step()

    def predict(self, x, y):# 忽略间隔
        size = x.size(0)
        w = self.w /self.w.norm()
        output = torch.matmul(x, w) + self.b
        accuracy = 0
        for i in range(size):
            if (output[i].sum() > 0 and int(y[i]) == 1) or (output[i].sum() < 0 and int(y[i]) == 0):
                accuracy += 1
        return accuracy / size

In [2]:
x_train_positive = torch.tensor(numpy.load("./train_x_positive.npy", allow_pickle=True))
x_train_negative = torch.tensor(numpy.load("./train_x_negative.npy", allow_pickle=True))
x_test = torch.tensor(numpy.load("./train_x.npy", allow_pickle=True))
y_test = torch.tensor(numpy.load("./train_y.npy", allow_pickle=True))

x_train_positive = x_train_positive.reshape(x_train_positive.shape[0], -1).cuda()
x_train_negative = x_train_negative.reshape(x_train_negative.shape[0], -1).cuda()
x_test, y_test = x_test.reshape(x_test.shape[0], -1).cuda(), y_test.cuda()
print(x_train_positive.shape)
print(x_train_negative.shape)
print(x_test.shape)
print(y_test.shape)

torch.Size([8400, 4096])
torch.Size([6960, 4096])
torch.Size([15360, 4096])
torch.Size([15360])


In [4]:
model = SVM().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

for i in range(150):# 训练与检测
    model.train()
    model(x_train_positive, x_train_negative, optimizer)

    model.eval()
    with torch.no_grad():
        pre = model.predict(x_test, y_test)
        print(f"Epoch{i + 1}, 准确率：{100 * pre :> 0.3f}%")

torch.save(model.state_dict(), './SVM.pth')

Epoch1, 准确率： 53.776%
Epoch2, 准确率： 50.436%
Epoch3, 准确率： 51.738%
Epoch4, 准确率： 51.979%
Epoch5, 准确率： 51.719%
Epoch6, 准确率： 53.151%
Epoch7, 准确率： 52.201%
Epoch8, 准确率： 53.516%
Epoch9, 准确率： 52.415%
Epoch10, 准确率： 54.121%
Epoch11, 准确率： 52.728%
Epoch12, 准确率： 54.635%
Epoch13, 准确率： 52.988%
Epoch14, 准确率： 55.163%
Epoch15, 准确率： 53.470%
Epoch16, 准确率： 55.260%
Epoch17, 准确率： 53.783%
Epoch18, 准确率： 55.547%
Epoch19, 准确率： 54.128%
Epoch20, 准确率： 55.664%
Epoch21, 准确率： 54.264%
Epoch22, 准确率： 55.514%
Epoch23, 准确率： 54.544%
Epoch24, 准确率： 55.749%
Epoch25, 准确率： 54.902%
Epoch26, 准确率： 55.749%
Epoch27, 准确率： 55.156%
Epoch28, 准确率： 55.905%
Epoch29, 准确率： 55.111%
Epoch30, 准确率： 55.879%
Epoch31, 准确率： 55.221%
Epoch32, 准确率： 55.951%
Epoch33, 准确率： 55.332%
Epoch34, 准确率： 55.944%
Epoch35, 准确率： 55.443%
Epoch36, 准确率： 55.820%
Epoch37, 准确率： 55.592%
Epoch38, 准确率： 55.814%
Epoch39, 准确率： 55.560%
Epoch40, 准确率： 55.755%
Epoch41, 准确率： 55.599%
Epoch42, 准确率： 55.762%
Epoch43, 准确率： 55.827%
Epoch44, 准确率： 55.690%
Epoch45, 准确率： 55.892%
Epoch46, 准确率： 55.63

In [ ]:
# Part.1
# 读取源数据，做基本处理，区分标签与数据
import random
import torch
import numpy
import pandas
from torcheeg import transforms
from torcheeg.datasets import DEAPDataset
from torcheeg.datasets.constants.emotion_recognition.deap import DEAP_CHANNEL_LIST
from torcheeg.datasets.constants.emotion_recognition.deap import DEAP_CHANNEL_LOCATION_DICT

dataset = DEAPDataset(io_path = f'../deap_CCNN', root_path = '../data_preprocessed_python',
                    io_mode = 'pickle',
                    offline_transform = transforms.Compose([
                        transforms.BandDifferentialEntropy(sampling_rate = 128, apply_to_baseline = True),
                        transforms.BaselineRemoval(),
                        transforms.ToGrid(DEAP_CHANNEL_LOCATION_DICT)
                    ]),
                    online_transform = transforms.ToTensor(),
                    label_transform = transforms.Compose([
                        transforms.Select('valence'),
                        transforms.Binary(5.0),
                    ]),
                    chunk_size = -1,
                    num_baseline = 3,
                    num_worker = 4)

# dataset = DEAPDataset(io_path=f'../deap_EEG', root_path='../data_preprocessed_python',
#             online_transform=transforms.Compose([
#                 transforms.To2d(),
#                 transforms.ToTensor(),
#             ]),
#             io_mode='pickle',
#             label_transform=transforms.Compose([
#                 transforms.Select('valence'),
#                 transforms.Binary(5.0),
#             ]))

# dataset = DEAPDataset(io_path='../deap_TSCeption', root_path='../data_preprocessed_python',
#                     io_mode='pickle',
#                     offline_transform=transforms.Compose([
#                         transforms.PickElectrode(transforms.PickElectrode.to_index_list(
#                         ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7',
#                         'PO3','O1', 'FP2', 'AF4', 'F4', 'F8', 'FC6', 'FC2', 'C4', 'T8', 'CP6', 'CP2',
#                         'P4', 'P8', 'PO4', 'O2'], DEAP_CHANNEL_LIST)),
#                         transforms.To2d()
#                     ]),
#                     online_transform=transforms.ToTensor(),
#                     label_transform=transforms.Compose([
#                         transforms.Select('valence'),
#                         transforms.Binary(5.0),
#                     ]))

for i, (train_dataset, val_dataset) in enumerate(dataset):
    if i % 1000 == 0:
        print(f'进度：{(100 * i / 76799) :> 0.3f}%')
    train_dataset = torch.unsqueeze(train_dataset, dim = 0).cuda()
    val_dataset = torch.unsqueeze(torch.tensor(val_dataset), dim = 0).cuda()
    if i == 0:
        x, y = train_dataset, val_dataset
    else:
        x = torch.cat((x, train_dataset), 0).cuda()
        y = torch.cat((y, val_dataset), 0).cuda()

x = x.detach().cpu().numpy()
y = y.detach().cpu().numpy()
numpy.save("./x_CCNN.npy", x)
numpy.save("./y_CCNN.npy", y)

# 按trail分类

data1 = list(pandas.read_csv("../deap_CCNN/info.csv")["trial_id"])
data2 = list(pandas.read_csv("../deap_EEG/info.csv")["trial_id"])

k = 0
Train_1, Train_2, Test_1, Test_2, Tuning_1, Tuning_2 = [], [], [], [], [], []
for i in range(len(data1)):
    if data1[i] < 16:
        Train_1.append(i)
    elif data1[i] < 24:
        Test_1.append(i)
    else:
        Tuning_1.append(i)
    if data2[i] < 16:
        Train_2.append(i)
    elif data2[i] < 24:
        Test_2.append(i)
    else:
        Tuning_2.append(i)

for _ in range(len(Train_1) - len(Test_1)):
    Test_1.append(-1)
    Test_2.append(-1)

pandas.DataFrame({"Train_1":Train_1, "Train_2":Train_2, "Test_1":Test_1, "Test_2":Test_2, 
                    "Tuning_1":Tuning_1, "Tuning_2":Tuning_2}).to_csv("../Trail_index.csv", index=False)

# 随机分类
train_arr = [i for i in range(76800)]

test_arr = random.sample(train_arr, 15360)
for i in test_arr:
    train_arr.remove(i)

tuning_arr = random.sample(train_arr, 30720)
for i in tuning_arr:
    train_arr.remove(i)
for _ in range(len(train_arr) - len(test_arr)):
    test_arr.append(-1)


pandas.DataFrame({"Train":train_arr, "Test":test_arr, "Tuning":tuning_arr}).to_csv("../Random_index.csv", index=False)

In [1]:
# 数据处理Part.2
# 对训练数据细分
import numpy
import torch
import pandas

arr = ["CCNN", "EEG", "TSC"]
mode = arr[1]

def split_easy(x, y, sign, gap = 16):
    x_out, y_out = None, []
    if sign == 1:
        index = list(pandas.read_csv("../Trail_index.csv")['Test_1'])[:15360]
    else:
        index = list(pandas.read_csv("../Trail_index.csv")['Test_2'])[:15360]

    # index = list(pandas.read_csv("../Random_index.csv")['Test'])[:15360]

    for i in range(gap):
        temp = None
        print(i + 1)
        for j in range(960):
            k = index[i * 960 + j]
            if temp == None:
                temp = torch.unsqueeze(x[k], dim = 0).cuda()
            else:
                temp = torch.cat((torch.unsqueeze(x[k], dim = 0).cuda(), temp), 0).cuda()

            y_out.append(int(y[k]) == 1)

        if i == 0:
            x_out = temp
        else:
            x_out = torch.cat((x_out, temp), 0).cuda()

    return x_out, torch.tensor(y_out), index

def data_presolve(x, y):
    lenth = int(x.shape[0] / 8)
    k = 0
    for i in range(8):# 一种很别扭的拆分
        print(i)
        temp_negative, temp_positive = None, None
        i_1, i_2 = 0, 0
        for k in range(i * lenth, (i + 1) * lenth):
            if int(y[k]) == 1:
                if i_1 == 0:
                    i_1 = 1
                    temp_positive = torch.unsqueeze(x[k], dim = 0).cuda()
                else:
                    temp_positive = torch.cat((torch.unsqueeze(x[k], dim = 0).cuda(), temp_positive), 0).cuda()
            else:
                if i_2 == 0:
                    i_2 = 1
                    temp_negative = torch.unsqueeze(x[k], dim = 0).cuda()
                else:
                    temp_negative = torch.cat((torch.unsqueeze(x[k], dim = 0).cuda(), temp_negative), 0).cuda()
        if i == 0:
            x_positive, x_negative = temp_positive, temp_negative
        else:
            x_positive, x_negative = torch.cat((x_positive, temp_positive), 0), torch.cat((x_negative, temp_negative), 0)

    return x_positive, x_negative

x = torch.tensor(numpy.load(f"./x_{mode}.npy", allow_pickle=True))
y = torch.tensor(numpy.load(f"./y_{mode}.npy", allow_pickle=True))

if mode == "CCNN":
    x_train, y_train, index = split_easy(x, y, 1)
else:
    x_train, y_train, index = split_easy(x, y, 2)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[2], -1)
y_train = y_train.reshape(y_train.shape[0])

print(x_train.shape)
print(y_train.shape)

train_x_positive, train_x_negative = data_presolve(x_train, y_train)# 做一个很烂的区分
numpy.save("./train_x_positive.npy", train_x_positive.detach().cpu().numpy())
numpy.save("./train_x_negative.npy", train_x_negative.detach().cpu().numpy())
numpy.save("./train_x.npy", x_train.detach().cpu().numpy())
numpy.save("./train_y.npy", y_train.detach().cpu().numpy())


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
torch.Size([15360, 32, 128])
torch.Size([15360])
0
1
2
3
4
5
6
7


In [11]:
# 记录距离，分类标签与实际标签
model = SVM().cuda()
model.load_state_dict(torch.load('./SVM.pth'))

x = torch.tensor(numpy.load("./train_x.npy", allow_pickle=True)).cuda()
x = x.reshape(x.shape[0], -1).cuda()
y = torch.tensor(numpy.load("./train_y.npy", allow_pickle=True)).cuda()

b, w = model.b, model.w
output = torch.matmul(x, w) + b

distance, classify_label, real_label = [], [], []
for i in range(output.shape[0]):
    distance.append(int(output[i].sum()))
    if int(y[i]) == 1:
        real_label.append(1)
    else:
        real_label.append(0)
    if distance[i] > 0:
        classify_label.append(1)
    else:
        classify_label.append(0)

pandas.DataFrame({'index':index, 'distance':distance, 'classify_label': classify_label, 'real_label': real_label}).to_csv(f"./SVM_{mode}_distance.csv", index = False)

In [12]:
dis = pandas.read_csv(f"./SVM_{mode}_distance.csv")

index, distance = dis["index"], dis["distance"]
arr = []

for i in range(len(index)):
    arr.append((index[i], distance[i]))

index, distance = [], []
arr = sorted(arr, key = lambda x : abs(x[1]))

for i in arr:
    index.append(i[0])
    distance.append(i[1])

pandas.DataFrame({'index':index, 'distance':distance}).to_csv(f"../SVM_{mode}_index.csv", index = False)